# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Libs utilizadas:
- Selenium
- Pandas

In [1]:
# Imports

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime


### Agora vamos atualiza a nossa base de preços com as novas cotações

#### Passo 1: Coletar as novas cotações

In [ ]:
# Criar o navegador
navegador = webdriver.Edge()
navegador.maximize_window()


# Pesquisar por "Cotação Dólar"
navegador.get('https://google.com')  # Entrar no Google
search_box = navegador.find_element(By.XPATH, "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input")
search_box.send_keys('Cotação Dólar', Keys.ENTER)

# Pegar a cotação do Dólar
cotacao_dolar = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]')
cotacao_dolar = cotacao_dolar.get_attribute('data-value')


# Pesquisar por "Cotação Euro"
navegador.get('https://google.com')
search_box = navegador.find_element(By.XPATH, "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input")
search_box.send_keys('Cotação Euro', Keys.ENTER)

# Pegar a cotação do Euro
cotacao_euro = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]')
cotacao_euro = cotacao_euro.get_attribute('data-value')


# Entrar no site Melhor Cambio
navegador.get('https://www.melhorcambio.com/ouro-hoje')

# Pegar a cotação do Ouro
cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute('value')

navegador.close()


print(cotacao_dolar)
print(cotacao_euro)
print(cotacao_ouro)

#### Passo 2: Importar a base de dados

In [ ]:
tabela = pd.read_excel('Produtos.xlsx')

display(tabela)

### Atualizando os preços e o cálculo do Preço Final

In [ ]:
# Atualizar as cotações

# Dolar
tabela.loc[tabela['Moeda'] == 'Dólar', 'Cotação'] = cotacao_dolar

# Euro
tabela.loc[tabela['Moeda'] == 'Euro', 'Cotação'] = cotacao_euro

# Ouro
tabela.loc[tabela['Moeda'] == 'Ouro', 'Cotação'] = cotacao_ouro.replace(',', '.')

# Corrigir a tabela
tabela['Cotação'] = pd.to_numeric(tabela['Cotação'])

# Atualizar Preço de Compra (Preço de Compra = Preço Original * Cotação)
tabela['Preço de Compra'] = tabela['Preço Original'] * tabela['Cotação']

# Atualizar Preço de venda (Preço de Venda = Preço de Compra * Margem)
tabela['Preço de Venda'] = tabela['Preço de Compra'] * tabela['Margem']

display(tabela)

### Agora vamos exportar a nova base de preços atualizada

In [ ]:
date = datetime.now()
data = f'{date.day}-{date.month}-{date.year}-{date.hour}-{date.minute}-{date.second}'
name = f'Produtos_atualizados_{data}.xlsx'

tabela.to_excel(name, index=False, startrow=0, sheet_name='Produtos')